<div id="instructions" style="border-radius: 5px; background-color:#f5f5f5;" >
<h1>Instructions</h1>
<p>Look for the <b>3 Your Turn</b> sections to complete the code and/or answer questions.<p>
</div>

# Lesson 4 - Sampling and Law of Large Numbers

## Introduction

Sampling is a fundamental process in the collection of data. Sampling is important because we can almost never look at the whole population. Some key points to keep in mind about sampling are:
- An understanding of sampling is essential to ensure that analyses performed are representative of the entire population. 
- You will use inferences on the sample to say something about the population.
- You will need estimates of variances on the sample calculations to say something about the population.

Let's look at some examples of sampling.

| Use Case | Sample | Population |
|---|---|---|
| A/B Testing | The users we show either web sites A or B | All possible users, past present and future|
|World Cup Soccer | 32 teams which qualify in one season | All national teams in past, present and future years|
|Average height of data science students | Students in UW Data Science Program | All students taking data science classes world wide|

Notice, that in several cases it is not only impractical, but impossible to collect data from the entire population. Hence, we must work with correctly collected samples. 

## Sampling Strategies

At the conclusion of this lesson, you should be able to apply a number of commonly used sampling strategies: 

- Bernoulli Sampling
- Stratified Sampling
- Cluster Sampling
- Systematic Sampling


### Bernoulli Sampling

Bernoulli sampling has the following properties:

- All population members have equal an chance of being part of the sample. 
- The sample sizes in Bernoulli sampling are not fixed, because each sample is considered separately. 
An example would be to randomly products in a factory to ensure quality.
Let's look at an example. The code in the cell below creates a data frame with 200 random samples of a normal distribution, divided into 4 groups. Run this code and examine the summary of the sample. 

In [ ]:
import numpy.random as nr
import pandas as pd
nr.seed(345)
data = pd.DataFrame({"var":nr.normal(size = 200), 
                     "group":nr.choice(range(4), size= 200, p = [0.1,0.3,0.4,0.2])})
data.head(10)

In [ ]:
def count_mean(dat):
    import numpy as np
    import pandas as pd
    groups = dat.groupby('group') # Create the groups
    ## Create a data frame with the counts and the means of the groups
    return pd.DataFrame({'count': groups.size(), 
                        'mean': groups.aggregate(np.mean).loc[:, 'var']})
count_mean(data)

Next, Bernoulli sample these data by running the code in the cell below.  

In [ ]:
p = 0.1
bernoulli = data.iloc[nr.choice(range(200), size = int(p * 200)), :]
count_mean(bernoulli)

You can see that the total number of samples is approximately 10%. Further, you can see that some of the mean estimates are further from zero.   

## Your Turn 1
You can also use the numpy.random `choice` method to Bernoulli sample a data frame. Create the code to Bernoulli **sample 20 cases** from the population and print the summary statistics of number and mean by group. 

Examine these results. Do the numbers of samples appear representative of the population? 

Notice in both cases you have tried, in both samples the number of cases for some groups can be rather small. Compare the means by groups of the two samples and the population, and notice how much variation there is. Clearly, Bernoulli sampling is far from ideal if the data have a group structure.  

### Stratified Sampling

Stratified sampling strategies are used when data are organized in groups or **Strata**. The idea is simple: sample each group in such a way that the sample includes a representative number of cases from each group. The simplest version of stratified sampling creates a sample with the same number of cases from each group.

Some examples of stratified sampling include:

- Sample equal numbers of men and women.
- Sample equal numbers of people in different income categories.
- Sample equal numbers of people from towns of different sizes.

Execute the code in the cell below to sample our population uniformly, and verify the result. The comments can help you understand the code. 

In [ ]:
p = 0.05
def stratify(dat, p):
    groups = dat.groupby('group') # Create the groups
    nums = min(groups.size()) # Find the size of the smallest group
    num = int(p * dat.shape[0]) # Compute the desired number of samples per group
    if num <= nums: # If sufficient group size, sample each group and return. 
        return groups.apply(lambda x: x.sample(n=num))
    else: # Oops. p is to large
        pmax = nums / dat.shape[0]
        print('The maximum value of p = ' + str(pmax))


In [ ]:
stratified = stratify(data, p)

In [ ]:
stratified.reset_index(inplace=True, drop=True)
count_mean(stratified)

We now have 5 samples from each strata. Notice that the mean estimates of each group are closer those for the population. Stratified sampling ensures the samples for each group are more representative of the groups. 

### Cluster Sampling

When sampling is expensive, a strategy is required to reduce the cost, yet still keep randomized sampling. Some examples of data that is expensive to collect includes:

- Surveys of customers at a chain of stores
- Door-to-door survey of homeowners

In these cases, the population can be divided into clusters and a random selection of clusters sampled. The process of cluster sampling follows these steps:

- Define the clusters and divide the data.
- Randomly select some clusters.
- Sample from the selected clusters.
- Optionally, stratify the sample from the clusters.

As an example, you can select a few store locations and Bernoulli sample customers at these locations.

The code in the cell bellow divides a population into clusters, randomly selects 3 clusters, and computes and prints some summary statistics. Run this code and examine the results.

In [ ]:
## First compute the clusters
num_clusters = 10
num_vals = 200
## Create a data frame with randomly sampled cluster numbers
clusters = pd.DataFrame({'group': range(num_clusters)}).sample(n = num_vals, replace = True)
## Add a column to the data frame with Normally distributed values
clusters.loc[:, 'var'] = nr.normal(size = num_vals)
clusters.head(10) # Print the head to get a feel for these data

So that we have a basis of comparison, we will compute the count and mean of each group in our population. 

In [ ]:
count_mean(clusters)

Examine these results. Notice that the number of samples in each group, and that the means are close to, but never exactly, zero. 

Next we randomly sample 3 of the clusters. 

In [ ]:
## Randomly sample the group numbers, making sure we sample from 
## unique values of the group numbers. 
clusters_samples = nr.choice(clusters.loc[:, 'group'].unique(), 
                             size = 3, replace = False)
## Now sample all rows with the selected cluster numbers
clus_samples = clusters.loc[clusters.loc[:, 'group'].isin(clusters_samples), :]
print('cluster sampled are: ')
[print(x) for x in clusters_samples] 
clus_samples.head(10)

You can see the  3 clusters of 20 cases each which have been selected.

Recalling that the population contains groups the sample from the clusters can be summarized by group by running the code in the cell below. 

In [ ]:
count_mean(clus_samples)

## Your Turn 2
In many cases, cluster sampling is often combined with random sampling to reduce the size of the sub-samples. Reducing the sample sizes can be important if the clusters are large or if the collection of data is expensive. 

Create a stratified sample of the three clusters with `p = 0.1` can then compute and display the count of the samples and the mean using the `count_mean` function. 

In [ ]:
cluster_stratified = stratify(                 )  #fill in the parameters
count_mean(cluster_stratified)

Examine the results. Are these samples reasonably representative of the full cluster samples and why? 

### Systematic Sampling

**WARNING: systematic sampling is a form of convenience sampling. Convenience sampling almost always leads to problems!**

In systematic sampling every kth case of the population is selected. As you can imagine, this approach is not a random sampling method, but rather a case of convenience sampling. 

The code in the cell bellow systematically samples the population by simply taking every 5th value as a sample. Run this code and examine the results. 

In [ ]:
## Systematic sampling every kth item.
k = 5
sys_sample = data.iloc[range(0, data.shape[0], k), :]
sys_sample.head(10)

Next, compute the number of samples and the means of each group based on these samples. Execute the code in the cell below and examine the results.

In [ ]:
count_mean(sys_sample)

Notice the differing sizes of the sample in each group and the bias in the mean estimates. This illustrates the hazards of convenience sampling. 

### A few more thoughts on sampling:

- Whenever you are planing to sample data, make sure you have a clear sampling plan. 
- Know number of clusters, strata, samples in advance.
- Don’t stop sampling when desired result is achieved: e.g., error measure! 
- Note that random sampling, if done properly, controls for database effects, like indexing. 


## Sampling and the Law of Large Numbers

The **law of large numbers** is a theorem that states that statistics of independent samples converge to the population values as additional unbiased experiments are performed. We can write this mathematically for the **expected value** of the mean as:

$$Let\ \bar{X} = \frac{1}{n}\sum_{i=1}^{n} X_i\\
then\ by\ the\ law\ of\ Large\ Numbers\\
\bar{X} \rightarrow E(X) = \mu\\
as\\
n \rightarrow \infty$$

The law of large numbers is foundational to statistics. We rely on the law of large numbers whenever we work with samples of data. We can rest assured that larger samples will be more representatives of the population we are sampling. This theorem is the foundation of not only sampling theory, but modern computation methods including, simulation, bootstrap resampling, and Monte Carlo methods. If the real world did not follow this theorem much of statistics, to say nothing of science and technology, would fail badly. 

Given this great importance, it should not be surprising that the law of large numbers has a long history. Jacob Bernoulli posthumously published the first proof for the Binomial distribution in 1713. In fact the law of large numbers is sometimes referred to as **Bernoulli's theorem**. A more general proof was published by Poisson in 1837.

Let's think about a simple example. The mean of 50 coin flips (0,1)=(T,H) is usually farther away from the true mean of 0.5 than the mean of 5,000 coin flips. The code in the cell below computes and then samples a population of 1,000,000 Bernoulli trials with $p=0.5$. Run this code and  examine the results. Does the expected value, or mean converge to 0.5 as `n` increases?

In [ ]:
nr.seed(3457)
n = 1
p = 0.5
size = 1000000
# Create a large binomial distributed population. 
pop = pd.DataFrame({'var':nr.binomial(n, p, size)}) 
# Sample the population for different sizes and compute the mean
out = [pop.sample(n = x).mean(axis = 0) for x in [5, 50, 500, 5000]] 
for x in out: print("%.2f" % x)

### Convergence of Bernoulli Trials 

Next, let's use the series of Bernoulli trials to examine the convergence of the estimated probability, $p$, for a number of flips of a fair coin. This series should converge to $p = 0.5$ as the number of samples increases. The code in the cell below computes the running mean of the realization of Bernoulli trials as the sample size increases. Run this code and examine the results. 

In [ ]:
# Calculate and plot a running average of N-trials of flipping a fair coin
import numpy as np
num_samps = 5000
running_average = pd.DataFrame({'num_samples': range(1, num_samps), 
                                'mean':np.concatenate([np.mean(pop.sample(n = x)) for x in range(1, num_samps)]).ravel().tolist()}) 

Now, run the code in the cell below to plot $p$ for the different sample sizes. Do the results appear to be converging to $p$ as $n \rightarrow \infty$.

In [ ]:
%matplotlib inline
def plot_samples(df, num_plot):
    import matplotlib.pyplot as plt
    running_average.iloc[:num_plot, :].plot(x = 'num_samples', y = 'mean')
    plt.ylabel('Mean estimate')
    plt.xlabel('Number of samples')
    plt.title('Mean estimate vs. number of samples')
plot_samples(running_average, 500)
plot_samples(running_average, 5000)

### Convergance for a Normal Distribution

As we sample from a normal distribution, the mean of the sample will converge to the population mean and the sample standard deviation will converge to the population standard deviation. The standard error of the sample mean should converge as:

$$se = \frac{sd}{\sqrt(n)}$$

The code in the cell below computes the mean and standard error of the mean estimate for 10 to 10,000 samples of a normal distribution. Execute this code.

In [ ]:
## Plot means and SE for 10000 Normal distributions
import math
start = 10
end = 10000
step = 10
norms = pd.DataFrame({'num_samples':range(start, end, step),
                      'mean': [np.mean(nr.normal(size = x)) for x in range(start, end, step)],
                       'se_up':[1/math.sqrt(n) for n in range(start, end, step)],
                       'se_dn':[-1/math.sqrt(n) for n in range(start, end, step)]})

Run the code in the cell below to plot the convergence of the sample mean as the number of samples increases.

In [ ]:
def plot_se(df):
    import matplotlib.pyplot as plt
    ax = plt.figure(figsize=(6, 6)).gca() # define axis
    df.plot.scatter(x = 'num_samples', y = 'mean', alpha = 0.2, style="o", ax = ax)
    df.plot(x = 'num_samples', y = 'se_up', c = 'red', ax = ax) 
    df.plot(x = 'num_samples', y = 'se_dn', c = 'red', ax = ax) 
    ax.set_ylabel('Mean estimate')
    ax.set_xlabel('Number of samples')
    ax.set_title('Mean estimate vs. number of samples')
plot_se(norms)    

## Your Turn 3
In the cell below, create and execute code to plot the convergence of the sample standard deviation. Try to include lines showing the expected bounds on the standard error. **Hint:** you will need to add `1.0` to the standard error of the sample standard deviation. 

In [ ]:
sds = pd.DataFrame(                     )#fill in the parameters

def plot_sd(df):
    import matplotlib.pyplot as plt
    ax = plt.figure(figsize=(6, 6)).gca() # define axis
    print(df.columns)
    df.plot.scatter(x = 'num_samples', y = 'sd', alpha = 0.2, style="o", ax = ax)
    df.plot(x = 'num_samples', y = 'se_up', c = 'red', ax = ax) 
    df.plot(x = 'num_samples', y = 'se_dn', c = 'red', ax = ax) 
    ax.set_ylabel('Standard deviation estimate')
    ax.set_xlabel('Number of samples')
    ax.set_title('Standard deviation estimate vs. number of samples')
plot_sd(sds)   

Examine this result. Does the estimate of the standard deviation appear to converge as expected as the number of samples increase? 

## Summary

In the lesson you have explored the concepts of sampling and how sampling relates to the law of large numbers. Nearly all the data we work with as data scientists is sampled from larger populations. It is therefore important to have an understanding of sampling methods and the convergence of statistical estimates. 

Specifically in this lesson you have done the following:
- Used Bernoulli sampling to generate randomized samples of a population.
- Applied stratified sampling to populations with unequal numbers of members. 
- Used cluster sampling to reduce the number of samples required for cases where data collection is expensive.
- Investigated the convergence of sample estimated statistics as the sample size grows. This convergence is referred to as *The Law of Large Numbers*. 

<div id="reminder" style="border-radius: 5px; background-color:#f5f5f5;" >
<h3>Reminder</h3>
<p>Use this notebook to answer the quiz questions related to the <b>Your Turn</b> sections.<p>
</div>